### Word Group Prepare For Youtube Tool

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "Portuguese"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = Portuguese
Lang Id = 5
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group Prepare For Youtube Tool"

Path(path).mkdir(parents=True, exist_ok=True)

#### Read Data

In [6]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [7]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(700)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic,https://www.youtube.com/watch?v=RIdNWADMmic&t=...
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs,https://www.youtube.com/watch?v=Wb5154ZxiHs&t=...
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE,https://www.youtube.com/watch?v=98B7hGXu9YE&t=...
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo,https://www.youtube.com/watch?v=TQTjOYzo9bo&t=...
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves,https://www.youtube.com/watch?v=TKPATf95Ves&t=...
...,...,...,...,...,...,...
411,a o,5808,5814,a o,cP3qQB196wA,https://www.youtube.com/watch?v=cP3qQB196wA&t=...
412,o a,430,436,o a,0hnzJc8PrPE,https://www.youtube.com/watch?v=0hnzJc8PrPE&t=...
413,o a,124,130,o a,svq7KDrLrwA,https://www.youtube.com/watch?v=svq7KDrLrwA&t=...
414,o a,128,134,o a,9qMisZZbKNI,https://www.youtube.com/watch?v=9qMisZZbKNI&t=...


In [8]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(700)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,o seu tempo porque eu não tenho nada de dinhei...,2049,2056,o seu tempo porque eu não tenho nada de dinhei...,2wA_t6MXdcU,https://www.youtube.com/watch?v=2wA_t6MXdcU&t=...
1,quer para nós vou fazer uma coisa que ela não ...,84,89,quer para nós vou fazer uma coisa que ela não ...,cm9OuklSuZ4,https://www.youtube.com/watch?v=cm9OuklSuZ4&t=84s
2,você pode fazer assim um por favor faz isso pa...,1126,1132,você pode fazer assim um por favor faz isso pa...,5HsmMvokixI,https://www.youtube.com/watch?v=5HsmMvokixI&t=...
3,de tempo ele já foi a coisa mais espera agora ...,809,814,um lapso de tempo ele já foi a coisa mais espe...,GcYPFyk5NAk,https://www.youtube.com/watch?v=GcYPFyk5NAk&t=...
4,com a minha mulher é como é porque eu tenho qu...,2926,2930,a relação com a minha mulher é como é porque e...,61FQdHgTyhA,https://www.youtube.com/watch?v=61FQdHgTyhA&t=...
...,...,...,...,...,...,...
695,mas todos nós temos essa,1060,1063,mas todos nós temos essa visão,MZH5gedZU8U,https://www.youtube.com/watch?v=MZH5gedZU8U&t=...
696,vamos ver como vai ficar,187,189,vamos ver como vai ficar,W_nt1g1jnok,https://www.youtube.com/watch?v=W_nt1g1jnok&t=...
697,bem está tudo bem senhor,1745,1747,bem está tudo bem senhor,cJsAdsdRuMM,https://www.youtube.com/watch?v=cJsAdsdRuMM&t=...
698,quero ver quem vai ficar,33,35,quero ver quem vai ficar,jOJZRyOo9WU,https://www.youtube.com/watch?v=jOJZRyOo9WU&t=33s


In [9]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(700)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,de qualquer forma eu tenho certeza que você ou...,58,68,de qualquer forma eu tenho certeza que você ou...,Ng85w0W2lRo,https://www.youtube.com/watch?v=Ng85w0W2lRo&t=58s
1,e ela dá a ideia de que ah eu não tenho nada a...,1189,1197,e ela dá a ideia de que ah eu não tenho nada a...,5yYk7YTnKdo,https://www.youtube.com/watch?v=5yYk7YTnKdo&t=...
2,nosso bebé pela primeira vez nem sempre aconte...,152,159,nosso bebé pela primeira vez nem sempre aconte...,qpm0lpb0iuI,https://www.youtube.com/watch?v=qpm0lpb0iuI&t=...
3,então de alguma forma eu tinha que fazer isso ...,122,130,então de alguma forma eu tinha que fazer isso ...,Eq7Ntuz42Vo,https://www.youtube.com/watch?v=Eq7Ntuz42Vo&t=...
4,do ponto de vista assim é que a gente vai até ...,581,587,depende do ponto de vista assim é que a gente ...,ASK6PbmZNq4,https://www.youtube.com/watch?v=ASK6PbmZNq4&t=...
...,...,...,...,...,...,...
695,ao minha casa minha vida são as grandes,6178,6181,ao minha casa minha vida são as grandes torres,GDIqqlhdk78,https://www.youtube.com/watch?v=GDIqqlhdk78&t=...
696,aí quando eles foram embora no final do,662,665,aí quando eles foram embora no final do turno,8zhCSCu3k7k,https://www.youtube.com/watch?v=8zhCSCu3k7k&t=...
697,toda família fica doente por causa dela,15,19,toda família fica doente por causa dela,eo9tTbdN_T0,https://www.youtube.com/watch?v=eo9tTbdN_T0&t=15s
698,estar entre estas duas grandes mulheres,37,41,estar entre estas duas grandes mulheres,IXFMyS3MyP8,https://www.youtube.com/watch?v=IXFMyS3MyP8&t=37s


In [10]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(700)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
1,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
2,vou explicar isso para você está é uma coisa b...,278,286,vou explicar isso para você está é uma coisa b...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
3,o japão finalmente tinha se tornado um exemplo...,369,377,o japão finalmente tinha se tornado um exemplo...,0ZQCwWbPbN4,https://www.youtube.com/watch?v=0ZQCwWbPbN4&t=...
4,nesse caso você vai colocar em cima do dedo as...,621,630,nesse caso você vai colocar em cima do dedo as...,Z7_CtSVJBmY,https://www.youtube.com/watch?v=Z7_CtSVJBmY&t=...
...,...,...,...,...,...,...
695,ganhei vários vou deixar esse pra tia oh ah ót...,1151,1157,ganhei vários vou deixar esse pra tia oh ah ót...,zWh5XLojPEQ,https://www.youtube.com/watch?v=zWh5XLojPEQ&t=...
696,negócio pronto aí ele fechar ficava assim num ...,994,999,negócio pronto aí ele fechar ficava assim num ...,eXpLKcLNsYY,https://www.youtube.com/watch?v=eXpLKcLNsYY&t=...
697,pela vida pelo ser humano pela natureza por al...,4769,4773,pela vida pelo ser humano pela natureza por al...,JuD5UgqFcwQ,https://www.youtube.com/watch?v=JuD5UgqFcwQ&t=...
698,aliás nós falamos tanto das coisas boas como t...,190,195,aliás nós falamos tanto das coisas boas como t...,XqjsdAQ_9d4,https://www.youtube.com/watch?v=XqjsdAQ_9d4&t=...


In [11]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(700)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
2,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
3,adoraria ter o problema é o valor dele né pess...,302,314,adoraria ter o problema é o valor dele né pess...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
4,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
...,...,...,...,...,...,...
695,pessoal olha muita gente me liga faz ligação m...,643,648,pessoal olha muita gente me liga faz ligação m...,X55aUW29VkE,https://www.youtube.com/watch?v=X55aUW29VkE&t=...
696,transmissão ao vivo nessa quarentena vimos mui...,697,702,transmissão ao vivo nessa quarentena vimos mui...,J5FyT3JkHWc,https://www.youtube.com/watch?v=J5FyT3JkHWc&t=...
697,tem alguns outros nome também todos sempre ass...,33,36,tem alguns outros nome também todos sempre ass...,BG4EkO6GzqU,https://www.youtube.com/watch?v=BG4EkO6GzqU&t=33s
698,nós pensamos primeiro eles vão descobrir sobre...,796,799,nós pensamos primeiro eles vão descobrir sobre...,GiebF1GsdkU,https://www.youtube.com/watch?v=GiebF1GsdkU&t=...


In [12]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(700)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o,https://www.youtube.com/watch?v=BampggOg28o&t=...
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE,https://www.youtube.com/watch?v=mAHS9bghaLE&t=...
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ,https://www.youtube.com/watch?v=fvtbRqPsTtQ&t=40s
...,...,...,...,...,...,...
695,trabalhar hoje esse sinal nós também costumamo...,57,66,trabalhar hoje esse sinal nós também costumamo...,5ye9JDVfkYI,https://www.youtube.com/watch?v=5ye9JDVfkYI&t=57s
696,aprendemos lições muito importantes vamos comp...,254,259,aprendemos lições muito importantes vamos comp...,N7COsyBF5Uk,https://www.youtube.com/watch?v=N7COsyBF5Uk&t=...
697,dos relatos mesmo professor esse menino tem al...,618,623,dos relatos mesmo professor esse menino tem al...,RatMkMfKLsw,https://www.youtube.com/watch?v=RatMkMfKLsw&t=...
698,contudo esta prática há muito já havia sido co...,97,102,contudo esta prática há muito já havia sido co...,Y4cIOHUA0Qg,https://www.youtube.com/watch?v=Y4cIOHUA0Qg&t=97s


In [13]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(700)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,que você pode estar utilizando ai em locais re...,1237,1252,que você pode estar utilizando ai em locais re...,DB31ARhbwAM,https://www.youtube.com/watch?v=DB31ARhbwAM&t=...
2,mas também com os nossos parceiros da indústri...,639,650,mas também com os nossos parceiros da indústri...,ELSjYhhLR9g,https://www.youtube.com/watch?v=ELSjYhhLR9g&t=...
3,porque os velhos espinhos e folhas estavam se ...,553,566,porque os velhos espinhos e folhas estavam se ...,65QGNVrSKwI,https://www.youtube.com/watch?v=65QGNVrSKwI&t=...
4,recentemente eu estou dando foco nos assuntos ...,32,44,recentemente eu estou dando foco nos assuntos ...,aEPd1q1ibbE,https://www.youtube.com/watch?v=aEPd1q1ibbE&t=32s
...,...,...,...,...,...,...
695,pra longe dali pois reconheceu qual das duas p...,316,321,pra longe dali pois reconheceu qual das duas p...,lHBW85VUJoQ,https://www.youtube.com/watch?v=lHBW85VUJoQ&t=...
696,logística ou seja ele pode fazer finanças admi...,579,586,logística ou seja ele pode fazer finanças admi...,jVt6R4QjN4w,https://www.youtube.com/watch?v=jVt6R4QjN4w&t=...
697,tecnologia avançada eles também adaptam essas ...,498,503,tecnologia avançada eles também adaptam essas ...,4SjuPYfQlTg,https://www.youtube.com/watch?v=4SjuPYfQlTg&t=...
698,nós sempre vamos estar ao seu lado ah tá vou s...,5685,5696,nós sempre vamos estar ao seu lado ah tá vou s...,SMKBWqU0Cqs,https://www.youtube.com/watch?v=SMKBWqU0Cqs&t=...


#### End Time Reassign

In [14]:
df_28_reassign = df_28.copy() # important because when it doesn't copy, main df are also change.
df_200_reassign = df_200.copy()
df_1000_reassign = df_1000.copy()
df_5000_reassign = df_5000.copy()
df_10000_reassign = df_10000.copy()
df_20000_reassign = df_20000.copy()
df_40000_reassign = df_40000.copy()

In [15]:
df_28_reassign["end_time"] = df_28_reassign["start_time"] + 1
df_200_reassign["end_time"] = df_200_reassign["start_time"] + 1
df_1000_reassign["end_time"] = df_1000_reassign["start_time"] + 1
df_5000_reassign["end_time"] = df_5000_reassign["start_time"] + 1
df_10000_reassign["end_time"] = df_10000_reassign["start_time"] + 1
df_20000_reassign["end_time"] = df_20000_reassign["start_time"] + 1
df_40000_reassign["end_time"] = df_40000_reassign["start_time"] + 1

In [16]:
out_path = "/home/kurubal/Downloads"

In [52]:
df_28_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_200_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_1000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_5000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_10000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_20000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_40000_reassign.to_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)

In [ ]:
# !!!!! End Time Reassign files result check on youtube tool and change file with youtube tool file result as same name (old file should remove) !!!!! 

#### End Time Reassign Convert Orjinal (From Youtube Tool Result)

In [17]:
df_28_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx")
df_28_video_list = list(set(df_28_youtube_tool["video_id"]))

In [18]:
df_200_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx")
df_200_video_list = list(set(df_200_youtube_tool["video_id"]))

In [19]:
df_1000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx")
df_1000_video_list = list(set(df_1000_youtube_tool["video_id"]))

In [20]:
df_5000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx")
df_5000_video_list = list(set(df_5000_youtube_tool["video_id"]))

In [21]:
df_10000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx")
df_10000_video_list = list(set(df_10000_youtube_tool["video_id"]))

In [22]:
df_20000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx")
df_20000_video_list = list(set(df_20000_youtube_tool["video_id"]))

In [23]:
df_40000_youtube_tool = pd.read_excel(f"{out_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx")
df_40000_video_list = list(set(df_40000_youtube_tool["video_id"]))

##### Video List In video_id 

In [24]:
df_28_new = df_28[df_28["video_id"].isin(df_28_video_list)]
#df_28_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv", index=False)
df_28_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.xlsx", index=False)
df_28_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,isso mas o que é que os,5641,5644,diz isso mas o que é que os,RIdNWADMmic,https://www.youtube.com/watch?v=RIdNWADMmic&t=...
1,mas ele não é bem com a,2616,2619,mas ele não é bem com a gente,Wb5154ZxiHs,https://www.youtube.com/watch?v=Wb5154ZxiHs&t=...
2,mas como que isso é uma,259,261,mas como que isso é uma gíria,98B7hGXu9YE,https://www.youtube.com/watch?v=98B7hGXu9YE&t=...
3,e os não mas e se eu os,2994,2999,né e os não mas e se eu os,TQTjOYzo9bo,https://www.youtube.com/watch?v=TQTjOYzo9bo&t=...
4,o que que é isso se ele,1635,1638,então o que que é isso se ele,TKPATf95Ves,https://www.youtube.com/watch?v=TKPATf95Ves&t=...
...,...,...,...,...,...,...
411,a o,5808,5814,a o,cP3qQB196wA,https://www.youtube.com/watch?v=cP3qQB196wA&t=...
412,o a,430,436,o a,0hnzJc8PrPE,https://www.youtube.com/watch?v=0hnzJc8PrPE&t=...
413,o a,124,130,o a,svq7KDrLrwA,https://www.youtube.com/watch?v=svq7KDrLrwA&t=...
414,o a,128,134,o a,9qMisZZbKNI,https://www.youtube.com/watch?v=9qMisZZbKNI&t=...


In [25]:
df_200_new = df_200[df_200["video_id"].isin(df_200_video_list)]
#df_200_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv", index=False)
df_200_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.xlsx", index=False)
df_200_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,o seu tempo porque eu não tenho nada de dinhei...,2049,2056,o seu tempo porque eu não tenho nada de dinhei...,2wA_t6MXdcU,https://www.youtube.com/watch?v=2wA_t6MXdcU&t=...
1,quer para nós vou fazer uma coisa que ela não ...,84,89,quer para nós vou fazer uma coisa que ela não ...,cm9OuklSuZ4,https://www.youtube.com/watch?v=cm9OuklSuZ4&t=84s
2,você pode fazer assim um por favor faz isso pa...,1126,1132,você pode fazer assim um por favor faz isso pa...,5HsmMvokixI,https://www.youtube.com/watch?v=5HsmMvokixI&t=...
3,de tempo ele já foi a coisa mais espera agora ...,809,814,um lapso de tempo ele já foi a coisa mais espe...,GcYPFyk5NAk,https://www.youtube.com/watch?v=GcYPFyk5NAk&t=...
4,com a minha mulher é como é porque eu tenho qu...,2926,2930,a relação com a minha mulher é como é porque e...,61FQdHgTyhA,https://www.youtube.com/watch?v=61FQdHgTyhA&t=...
...,...,...,...,...,...,...
695,mas todos nós temos essa,1060,1063,mas todos nós temos essa visão,MZH5gedZU8U,https://www.youtube.com/watch?v=MZH5gedZU8U&t=...
696,vamos ver como vai ficar,187,189,vamos ver como vai ficar,W_nt1g1jnok,https://www.youtube.com/watch?v=W_nt1g1jnok&t=...
697,bem está tudo bem senhor,1745,1747,bem está tudo bem senhor,cJsAdsdRuMM,https://www.youtube.com/watch?v=cJsAdsdRuMM&t=...
698,quero ver quem vai ficar,33,35,quero ver quem vai ficar,jOJZRyOo9WU,https://www.youtube.com/watch?v=jOJZRyOo9WU&t=33s


In [26]:
df_1000_new = df_1000[df_1000["video_id"].isin(df_1000_video_list)]
#df_1000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv", index=False)
df_1000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.xlsx", index=False)
df_1000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,de qualquer forma eu tenho certeza que você ou...,58,68,de qualquer forma eu tenho certeza que você ou...,Ng85w0W2lRo,https://www.youtube.com/watch?v=Ng85w0W2lRo&t=58s
1,e ela dá a ideia de que ah eu não tenho nada a...,1189,1197,e ela dá a ideia de que ah eu não tenho nada a...,5yYk7YTnKdo,https://www.youtube.com/watch?v=5yYk7YTnKdo&t=...
2,nosso bebé pela primeira vez nem sempre aconte...,152,159,nosso bebé pela primeira vez nem sempre aconte...,qpm0lpb0iuI,https://www.youtube.com/watch?v=qpm0lpb0iuI&t=...
3,então de alguma forma eu tinha que fazer isso ...,122,130,então de alguma forma eu tinha que fazer isso ...,Eq7Ntuz42Vo,https://www.youtube.com/watch?v=Eq7Ntuz42Vo&t=...
4,do ponto de vista assim é que a gente vai até ...,581,587,depende do ponto de vista assim é que a gente ...,ASK6PbmZNq4,https://www.youtube.com/watch?v=ASK6PbmZNq4&t=...
...,...,...,...,...,...,...
695,ao minha casa minha vida são as grandes,6178,6181,ao minha casa minha vida são as grandes torres,GDIqqlhdk78,https://www.youtube.com/watch?v=GDIqqlhdk78&t=...
696,aí quando eles foram embora no final do,662,665,aí quando eles foram embora no final do turno,8zhCSCu3k7k,https://www.youtube.com/watch?v=8zhCSCu3k7k&t=...
697,toda família fica doente por causa dela,15,19,toda família fica doente por causa dela,eo9tTbdN_T0,https://www.youtube.com/watch?v=eo9tTbdN_T0&t=15s
698,estar entre estas duas grandes mulheres,37,41,estar entre estas duas grandes mulheres,IXFMyS3MyP8,https://www.youtube.com/watch?v=IXFMyS3MyP8&t=37s


In [27]:
df_5000_new = df_5000[df_5000["video_id"].isin(df_5000_video_list)]
#df_5000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv", index=False)
df_5000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.xlsx", index=False)
df_5000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
1,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
2,vou explicar isso para você está é uma coisa b...,278,286,vou explicar isso para você está é uma coisa b...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
3,o japão finalmente tinha se tornado um exemplo...,369,377,o japão finalmente tinha se tornado um exemplo...,0ZQCwWbPbN4,https://www.youtube.com/watch?v=0ZQCwWbPbN4&t=...
4,nesse caso você vai colocar em cima do dedo as...,621,630,nesse caso você vai colocar em cima do dedo as...,Z7_CtSVJBmY,https://www.youtube.com/watch?v=Z7_CtSVJBmY&t=...
...,...,...,...,...,...,...
695,ganhei vários vou deixar esse pra tia oh ah ót...,1151,1157,ganhei vários vou deixar esse pra tia oh ah ót...,zWh5XLojPEQ,https://www.youtube.com/watch?v=zWh5XLojPEQ&t=...
696,negócio pronto aí ele fechar ficava assim num ...,994,999,negócio pronto aí ele fechar ficava assim num ...,eXpLKcLNsYY,https://www.youtube.com/watch?v=eXpLKcLNsYY&t=...
697,pela vida pelo ser humano pela natureza por al...,4769,4773,pela vida pelo ser humano pela natureza por al...,JuD5UgqFcwQ,https://www.youtube.com/watch?v=JuD5UgqFcwQ&t=...
698,aliás nós falamos tanto das coisas boas como t...,190,195,aliás nós falamos tanto das coisas boas como t...,XqjsdAQ_9d4,https://www.youtube.com/watch?v=XqjsdAQ_9d4&t=...


In [28]:
df_10000_new = df_10000[df_10000["video_id"].isin(df_10000_video_list)]
#df_10000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv", index=False)
df_10000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.xlsx", index=False)
df_10000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
2,hoje a sua empresa pode dar atenção a um clien...,464,472,hoje a sua empresa pode dar atenção a um clien...,G8Ho4UQ3JoI,https://www.youtube.com/watch?v=G8Ho4UQ3JoI&t=...
3,adoraria ter o problema é o valor dele né pess...,302,314,adoraria ter o problema é o valor dele né pess...,4PumRaJ1fcc,https://www.youtube.com/watch?v=4PumRaJ1fcc&t=...
4,durante o dia e à noite a gente tem que carreg...,140,151,durante o dia e à noite a gente tem que carreg...,QqDQ37pGmrQ,https://www.youtube.com/watch?v=QqDQ37pGmrQ&t=...
...,...,...,...,...,...,...
695,pessoal olha muita gente me liga faz ligação m...,643,648,pessoal olha muita gente me liga faz ligação m...,X55aUW29VkE,https://www.youtube.com/watch?v=X55aUW29VkE&t=...
696,transmissão ao vivo nessa quarentena vimos mui...,697,702,transmissão ao vivo nessa quarentena vimos mui...,J5FyT3JkHWc,https://www.youtube.com/watch?v=J5FyT3JkHWc&t=...
697,tem alguns outros nome também todos sempre ass...,33,36,tem alguns outros nome também todos sempre ass...,BG4EkO6GzqU,https://www.youtube.com/watch?v=BG4EkO6GzqU&t=33s
698,nós pensamos primeiro eles vão descobrir sobre...,796,799,nós pensamos primeiro eles vão descobrir sobre...,GiebF1GsdkU,https://www.youtube.com/watch?v=GiebF1GsdkU&t=...


In [29]:
df_20000_new = df_20000[df_20000["video_id"].isin(df_20000_video_list)]
#df_20000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv", index=False)
df_20000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.xlsx", index=False)
df_20000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,a vida que tu tens levado é absolutamente méri...,136,146,a vida que tu tens levado é absolutamente méri...,BampggOg28o,https://www.youtube.com/watch?v=BampggOg28o&t=...
2,agora eu acho que você está bem preparada pra ...,840,848,agora eu acho que você está bem preparada pra ...,vwJq3pwexqc,https://www.youtube.com/watch?v=vwJq3pwexqc&t=...
3,louie exatamente então acho que é importante a...,439,446,louie exatamente então acho que é importante a...,mAHS9bghaLE,https://www.youtube.com/watch?v=mAHS9bghaLE&t=...
4,neste caso eu vou à empresa contrato uma pesso...,40,48,neste caso eu vou à empresa contrato uma pesso...,fvtbRqPsTtQ,https://www.youtube.com/watch?v=fvtbRqPsTtQ&t=40s
...,...,...,...,...,...,...
695,trabalhar hoje esse sinal nós também costumamo...,57,66,trabalhar hoje esse sinal nós também costumamo...,5ye9JDVfkYI,https://www.youtube.com/watch?v=5ye9JDVfkYI&t=57s
696,aprendemos lições muito importantes vamos comp...,254,259,aprendemos lições muito importantes vamos comp...,N7COsyBF5Uk,https://www.youtube.com/watch?v=N7COsyBF5Uk&t=...
697,dos relatos mesmo professor esse menino tem al...,618,623,dos relatos mesmo professor esse menino tem al...,RatMkMfKLsw,https://www.youtube.com/watch?v=RatMkMfKLsw&t=...
698,contudo esta prática há muito já havia sido co...,97,102,contudo esta prática há muito já havia sido co...,Y4cIOHUA0Qg,https://www.youtube.com/watch?v=Y4cIOHUA0Qg&t=97s


In [30]:
df_40000_new = df_40000[df_40000["video_id"].isin(df_40000_video_list)]
#df_40000_new.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv", index=False)
df_40000_new.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.xlsx", index=False)
df_40000_new

,search_string,start_time,end_time,sentence,video_id,video_url
0,mesmo se você não for cliente vale a pena acom...,51,77,mesmo se você não for cliente vale a pena acom...,MZrSIz9MUo4,https://www.youtube.com/watch?v=MZrSIz9MUo4&t=51s
1,que você pode estar utilizando ai em locais re...,1237,1252,que você pode estar utilizando ai em locais re...,DB31ARhbwAM,https://www.youtube.com/watch?v=DB31ARhbwAM&t=...
2,mas também com os nossos parceiros da indústri...,639,650,mas também com os nossos parceiros da indústri...,ELSjYhhLR9g,https://www.youtube.com/watch?v=ELSjYhhLR9g&t=...
3,porque os velhos espinhos e folhas estavam se ...,553,566,porque os velhos espinhos e folhas estavam se ...,65QGNVrSKwI,https://www.youtube.com/watch?v=65QGNVrSKwI&t=...
4,recentemente eu estou dando foco nos assuntos ...,32,44,recentemente eu estou dando foco nos assuntos ...,aEPd1q1ibbE,https://www.youtube.com/watch?v=aEPd1q1ibbE&t=32s
...,...,...,...,...,...,...
695,pra longe dali pois reconheceu qual das duas p...,316,321,pra longe dali pois reconheceu qual das duas p...,lHBW85VUJoQ,https://www.youtube.com/watch?v=lHBW85VUJoQ&t=...
696,logística ou seja ele pode fazer finanças admi...,579,586,logística ou seja ele pode fazer finanças admi...,jVt6R4QjN4w,https://www.youtube.com/watch?v=jVt6R4QjN4w&t=...
697,tecnologia avançada eles também adaptam essas ...,498,503,tecnologia avançada eles também adaptam essas ...,4SjuPYfQlTg,https://www.youtube.com/watch?v=4SjuPYfQlTg&t=...
698,nós sempre vamos estar ao seu lado ah tá vou s...,5685,5696,nós sempre vamos estar ao seu lado ah tá vou s...,SMKBWqU0Cqs,https://www.youtube.com/watch?v=SMKBWqU0Cqs&t=...


#### Copy Move And Delete

In [31]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_*_Word{file_ext}.*")
output_file2

['Portuguese_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx',
 'Portuguese_Word_Group_Max_In_Youtube_Sentence_40000_Word6.xlsx']

In [32]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [33]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass